In [1]:
!nvidia-smi

Sun Sep 15 18:11:52 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.06             Driver Version: 535.183.06   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       On  | 00000000:00:1E.0 Off |                    0 |
| N/A   45C    P0              26W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
import os

In [3]:
# Space I have 
!df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay         100G   44G   57G  44% /
tmpfs            64M     0   64M   0% /dev
tmpfs           7.7G     0  7.7G   0% /sys/fs/cgroup
/dev/nvme0n1p1  100G   44G   57G  44% /run
tmpfs            14G     0   14G   0% /dev/shm
/dev/nvme2n1    2.0G   35M  1.9G   2% /home/jovyan
tmpfs            14G  120K   14G   1% /home/jovyan/.saturn
tmpfs            14G   12K   14G   1% /run/secrets/kubernetes.io/serviceaccount
tmpfs           7.7G   12K  7.7G   1% /proc/driver/nvidia
tmpfs           7.7G  3.0M  7.7G   1% /run/nvidia-persistenced/socket
tmpfs           7.7G     0  7.7G   0% /proc/acpi
tmpfs           7.7G     0  7.7G   0% /sys/firmware


In [4]:
# Checking all the environment variables available
#os.environ

In [5]:
# As hugging face stores the file inside the cache folder inside HOME directory which in our case doesn't have mmuch storage, so changing the default storage of Hugging Face
# HF_HOME is the variable which contains the location to store the Hugging Face data
# MAKE SURE TO RESTART THE KERNEL AFTER THIS
os.environ['HF_HOME'] = '/run/cache/'

In [6]:
os.environ['HF_HOME']

'/run/cache/'

# Copied from Hugging Face <a href="https://huggingface.co/docs/transformers/en/model_doc/mistral"> Webage of the model</a> 

## NOTE: <span style="color:red"> To access the mistral model, first log in into the hugging space and then accept the agreement <a href="https://huggingface.co/mistralai/Mistral-7B-v0.1">here</a>, and then login into Hugging Face from this notebook</span>

In [7]:
# Logging into Hugging Face
from huggingface_hub import login
login(os.environ["HUGGING_FACE_TOKEN"])

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /run/cache/token
Login successful


## <span style="color:red"> Requirements as mentioned in the documentation to run this model

In [8]:
#!pip install git+https://github.com/huggingface/transformers.git
#!pip install git+https://github.com/huggingface/accelerate

## Loading of Model and Tokenizer

In [9]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [10]:
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-v0.1", device_map="auto", load_in_4bit=True)
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

You are calling `save_pretrained` to a 4-bit converted model, but your `bitsandbytes` version doesn't support it. If you want to save 4-bit models, make sure to have `bitsandbytes>=0.41.3` installed.


In [11]:
prompt = "My favourite condiment is"

model_inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

generated_ids = model.generate(**model_inputs, max_new_tokens=100, do_sample=True)
tokenizer.batch_decode(generated_ids)[0]

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
/opt/saturncloud/envs/saturn/lib/python3.9/site-packages/bitsandbytes/nn/modules.py:224: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.')
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


'<s> My favourite condiment is ketchup. There’s no arguing with me on this matter. And, as you know, there are ketchup lovers and ketchup haters in every family. Not mine! I have been successful so far in training everyone, even my husband, to love ketchup, except for my youngest daughter, and I’m not sure there’s any hope yet for her.\n\nI’m happy to eat ketchup on just about anything. On fried'

## When using Pipeline from Transformers and creating a generator

In [12]:
from transformers import pipeline
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [13]:
response = generator(prompt, max_length=500, temperature=0.7, top_p=0.95, num_return_sequences=1)
response_final = response[0]['generated_text']
print(response_final[len(prompt):].strip())

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/opt/saturncloud/envs/saturn/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:599: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/opt/saturncloud/envs/saturn/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:604: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  war

mayonnaise. I love it. I love it on sandwiches, I love it on burgers, I love it on fries, I love it on chicken, I love it on fish, I love it on pizza, I love it on salads, I love it on hot dogs, I love it on eggs, I love it on toast, I love it on pasta, I love it on rice, I love it on potatoes, I love it on vegetables, I love it on fruit, I love it on meat, I love it on cheese, I love it on bread, I love it on crackers, I love it on chips, I love it on pancakes, I love it on waffles, I love it on ice cream, I love it on cake, I love it on cookies, I love it on muffins, I love it on scones, I love it on muffins, I love it on pies, I love it on tarts, I love it on cakes, I love it on pastries, I love it on doughnuts, I love it on bagels, I love it on croissants, I love it on brioche, I love it on buns, I love it on rolls, I love it on breadsticks, I love it on baguettes, I love it on focaccia, I love it on ciabatta, I love it on pita, I love it on naan, I love it on tortillas, I love it 

# ===== <span style="color:blue">Combining everything together to form a RAG</span> =====

## <span style="color:red">NOTE: Be aware that the model, tokenizer, generator used in this combined part has been defined previously above</span>

In [14]:
# Getting the Similar Doc Search python file created in Part_01_Basics
!rm -f similarDocSearch.py
!wget https://raw.githubusercontent.com/vaibhavcodes/LLM_RAG/main/Part_01_Basics/similarDocSearch.py

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


--2024-09-15 18:13:50--  https://raw.githubusercontent.com/vaibhavcodes/LLM_RAG/main/Part_01_Basics/similarDocSearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3320 (3.2K) [text/plain]
Saving to: ‘similarDocSearch.py’

similarDocSearch.py 100%[===================>]   3.24K  --.-KB/s    in 0s      

2024-09-15 18:13:50 (46.6 MB/s) - ‘similarDocSearch.py’ saved [3320/3320]



In [15]:
from similarDocSearch import *
import json
import requests

In [16]:
# Documents fetching from the github
docs_url = "https://github.com/vaibhavcodes/LLM_RAG/blob/main/Part_01_Basics/data/documents.json?raw=1"
docs_response = requests.get(docs_url)
docs_raw = docs_response.json()
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course'] 
        documents.append(doc)

In [17]:
# Initializing the class similarDocSearch
obj = similarDocSearch(text_fields = ["question", "text", "section"])
# Fitting
obj.fit(documents)

In [18]:
# Function that would search the most similar documents to our query and will return it
def search(query):
    boost = {'question': 3, 
         'section': 0.5} # Used to give a field more or less importance 
    
    filter_course = "data-engineering-zoomcamp"
    num_results = 5
    
    results = obj.search(query = query, boosts=boost, filter_course=filter_course, num_results=num_results)
    return results

In [19]:
# Function that would build up the prompt
def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
    Use only the facts from the CONTEXT when answering the QUESTION.
    If the CONTEXT doesn't contains the answer, output None
    
    QUESTION: {question}
    
    CONTEXT: 
    {context}
    
    Based on the QUESTION and CONTEXT provide, please provide the ANSWER in only 200 words.
    """.strip()

    # Context creation to be passed into the prompt
    context = ""
    
    for doc in search_results:
        #context = context + f"section: {doc['section']}\n question: {doc['question']}\n answer: {doc['text']}\n \n"
        context = context + f"{doc['question']}\n {doc['text']}\n \n"
    
    # Prompt formatting
    prompt = prompt_template.format(question=query, context=context).strip()
    
    return(prompt)

In [20]:
# Function that would use LLM
def llm(prompt):
    model_inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
    generated_ids = model.generate(**model_inputs, max_new_tokens=100, do_sample=True)
    result = tokenizer.batch_decode(generated_ids)[0]
    return result 

In [21]:
# Function that would make a whole RAG
def RAG(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [22]:
q = "I just discovered the course. Can i still join it??"
print(RAG(q))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


<s> You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
    Use only the facts from the CONTEXT when answering the QUESTION.
    If the CONTEXT doesn't contains the answer, output None
    
    QUESTION: I just discovered the course. Can i still join it??
    
    CONTEXT: 
    Course - Can I still join the course after the start date?
 Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.
 
Course - When will the course start?
 The purpose of this document is to capture frequently asked
technical question
The next cohort starts in Jan 2025. More inFo at DTC Article.
Register before the course starts using this link.
Join the course Telegram channel with announcements.
Don’t forget to register in DataTalks.Club's Slack and join the channel.
 
Course - What can I do before the course s

#### The result obtained is not good, but using pipeline improves the result

## When using Pipeline from transformers- generator used here has been defined previously before combining all the code here

In [23]:
# Function that would use LLM
# generator defined before combining the code
def llm(prompt):
    response = generator(prompt, max_new_tokens=100, max_length=100, temperature=0.7, top_p=0.95, num_return_sequences=1)
    response_final = response[0]['generated_text']
    # As can be seen from previous output that model also gives the prompt and at the end gives the answer
    # So we are ignoring the prompt text via slicing
    result  = response_final[len(prompt):].strip()
    return result

In [24]:
# Function that would make a whole RAG
def RAG(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [25]:
q = "I just discovered the course. Can i still join it??"
print(RAG(q))

/opt/saturncloud/envs/saturn/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:599: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/opt/saturncloud/envs/saturn/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:604: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Both `max_new_tokens` (=100) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


?
    Answer:
    Yes, you can join the course after the start date.
    The course is open to new students until the end of the semester.
    After that, you can still join the course, but you will not be able to participate in the final exam.
    You will have to take the final exam at the end of the semester.
    You can join the course at any time during the semester.
    You can join the


# <span style="colore:green">To improve the output, play around with prompt and tuning the parameters</span>